In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate
import scipy.sparse as sp

In [ ]:
#constants
hbar, m = 1, 1
k0, L = 20, 20
E = (hbar * k0) ** 2 / 2 / m
omega = E / hbar


def psi_I_anal(x,x0,sigmax, t):
    return np.exp(-(x - x0)**2 / (2 * sigmax**2)) * np.sin(k0 * x - omega * t)
                  
def psi_R_anal(x,x0,sigmax, t):
    return np.exp(-(x - x0)**2 / (2 * sigmax **2)) * np.cos(k0 * x - omega * t)


def make_psi_init(x0,sigmax,Nx,dt,L):
    x = np.linspace(0, L, Nx)
    C = psi_R_anal(x,x0,sigmax,0) @ psi_R_anal(x,x0,sigmax,0) + psi_I_anal(x,x0,sigmax,0) @ psi_I_anal(x,x0,sigmax,0)
    psi_R_init = psi_R_anal(x,x0,sigmax, 0) / np.sqrt(C)
    psi_I_init = psi_I_anal(x,x0,sigmax, -dt/2) / np.sqrt(C)
    return x , psi_R_init, psi_I_init

In [ ]:
def timeEvolve(psi_R,psi_I,n,V,dt):
    Nx = len(V)
    dx = L / (Nx-1)
    
    D1 = V / hbar + hbar / (m * dx**2) * np.ones(Nx) 
    D2 = - hbar / (2 * m * dx**2) *  np.ones(Nx-1) 
    H = sp.diags([D2, D1 ,D2],[-1,0,1])
    for i in range(1,n):
        psi_I = psi_I - dt * H.dot(psi_R)
        psi_R = psi_R + dt * H.dot(psi_I)
    return psi_R,psi_I

        

## Excercise 2
By testing various time steps it was found that $\varDelta T = 0.5 \cdot \varDelta T_0 $ with $\varDelta T_0 := \frac{\hbar}{\frac{\hbar^2}{2m} \frac{1}{\varDelta X^2} + V_{max}}$ gave satisfactory numerical stability.

The group velocity can be calculated using the relation $v_g = \frac{\partial \omega}{\partial k} \rvert_{k_0}$ giving the result $v_g = \frac{\hbar k_0}{m}$. By propagating the wave for a time $\frac{l}{v_g}$ the wave packet will move a distance l. Figures (XX1) and (XX2) show the wave packet after propagating a distance $l = 10$ from the start position $x_0 = 5$ for the values $\sigma = 0.5$ and $\sigma = 1.5$. The wave packet with initial $\sigma = 1.5$ has a larger spread than the wave with inital $\sigma = 0.5$. The spread is not as large as for the initial condition, ilustrating the fact that waves with a narrower probability density spread faster. 

In [ ]:
x0_2 = 5
Nx_2 =1000
V_2 = np.zeros(Nx_2)
v_g = hbar * k0 / m
dt_0 = hbar / ((hbar**2/(2 * m))*((Nx_2-1)/L)**2)
dt_2 = 0.5* dt_0
n_2 = int(L/ 2 * m / (hbar * k0 * dt_2))
    
dt_0 = hbar / ((hbar**2/(2 * m))*((Nx_2-1)/L)**2)

sigmax_2 = 0.5
x, psi_R_init,psi_I_init = make_psi_init(x0_2,sigmax_2,Nx_2,dt_2,L)
psi_R , psi_I = timeEvolve(psi_R_init,psi_I_init,n_2,V_2,dt_2)
plt.plot(x,psi_R) #endre til sannsynlighetstetthet
plt.show()

sigmax_2 = 1.5
x, psi_R_init,psi_I_init = make_psi_init(x0_2,sigmax_2,Nx_2,dt_2,L)
psi_R , psi_I = timeEvolve(psi_R_init,psi_I_init,n_2,V_2,dt_2)
plt.plot(x,psi_R) #endre til sannsynlighetstetthet
plt.show()

In [ ]:
def plot_barrier(x, psi_init, psi_final, V):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(20, 8)
    
    ax1_2 = ax1.twinx()                           
    l1, = ax1.plot(x, V, color = "tab:red")
    l1_2, = ax1_2.plot(x, psi_init)                       
    
    ax2_2 = ax2.twinx()
    l2, = ax2.plot(x, V, color = "tab:red")                               
    l2_2, = ax2_2.plot(x, psi_final)                

In [ ]:
##Barrier, exercise 3
n = 100
L = 20
l = L / 50
b = E / 2.0
dt = 0.001

Vb = np.zeros(Nx)
Vk = np.ones(int(l / L * Nx)) * b
Vb[int(Nx/2 - l * Nx / L /  2): int(Nx/2 + l * Nx / L /  2) ] = Vk


x, psi_R_init, psi_I_init = make_psi_init(x0, sigmax, Nx, dt, L)

psi_R_b,psi_I_b = timeEvolve(psi_R_init, psi_I_init, n, V, dt)


plot_barrier(x, psi_R_init, psi_R_b, V)